# Fine-tuning the top layers of a a pre-trained network

"fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier

implement in 3 steps
instantiate the convolutional base of VGG16 and load its weights
add our previously defined fully-connected model on top, and load its weights
freeze the layers of the VGG16 model up to the last convolutional block

In [ ]:
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def batch_generator(X_train, Y_train, batch_size, steps):
    while True:
        for i in range(steps):
            start_pos = i*batch_size
            end_pos = (i+1)*batch_size
            # yield batch size samples when does not meet ends, else yield rest of samples
            if num_samples > (i * batch_size):
                X_train_batch = X_train[start_pos:end_pos, :, :, :]
                Y_train_batch = Y_train[start_pos:end_pos, :]
            else:
                X_train_batch = X_train[start_pos:, :, :, :]
                Y_train_batch = Y_train[start_pos:, :]  
            
            # Rescale features    
            X_train_batch = (X_train_batch / 255) * 2.0 - 1.0 
            
            yield X_train_batch, Y_train_batch

In [ ]:
# Load raw data 
X_train = np.load(".\\data\\X_train.npy")
Y_train = np.load(".\\data\\Y_train.npy")
X_test = np.load(".\\data\\X_test.npy")
Y_test = np.load(".\\data\\Y_test.npy")
print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

In [ ]:
# Rescale features
X_test = (X_test / 255) * 2.0 - 1.0 

In [ ]:
# path to the model weights files.
weights_path = 'vgg16_weights_aug.h5'
top_model_weights_path = 'bottleneck_fc_model_aug.h5'
# dimensions of our images.
img_width, img_height = 150, 150

num_samples = len(X_train)
epochs = 50
batch_size = 64
steps = math.ceil(num_samples / batch_size)

In [ ]:
train_generator = batch_generator(X_train, Y_train, batch_size, steps)

In [ ]:
# build the VGG16 network
input_tensor = Input(shape=(150,150,3))
base_model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [ ]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(25, activation='sigmoid'))

In [ ]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [ ]:
# add the model on top of the convolutional base
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
# add checkpoint to save best model
checkpoint = ModelCheckpoint(weights_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# check 5 epochs
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max')
callbacks_list = [checkpoint, early_stop]

In [ ]:
history = model.fit_generator(train_generator,
          steps_per_epoch = steps,
          epochs=epochs,
          callbacks=callbacks_list,
          validation_data=(X_test, Y_test))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.show()
plt.savefig("fine_tune_VGG_acc_history_aug.png")

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.show()
plt.savefig("fine_tune_VGG_loss_history_aug.png")

In [ ]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("fine_tune_VGG_model_aug.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)